# Lab | API wrappers - Create your collection of songs & audio features (GNOD - part 3)

In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                           client_secret='<your client secret here>'))

In [2]:
secrets_file = open("secret.txt","r")

In [3]:
string = secrets_file.read()

In [4]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['CDI'],
                                                           client_secret=secrets_dict['Secret']))

In [6]:
# we will use our 'first paid music' playlist as an example:
playlist = sp.user_playlist_tracks("spotify", "32lPilIPcRVIDxlMdqEsCQ")

In [7]:
# Let's look at items and total:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [8]:
playlist["total"] 

1212

In [9]:
#first we need to get a list of individual tracks from the playlist
def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

In [10]:
def get_name_artists_from_playlist(input_playlist):
    return list(map(get_name_artist_from_playlist_item,input_playlist["items"]))

In [11]:
def get_name_artists_from_playlist(input_playlist):
    return flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

# Function to extract all songs from a playlist


In [12]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [13]:
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
first_playlist=get_playlist_tracks("32lPilIPcRVIDxlMdqEsCQ")
## importing list 

In [ ]:
len(first_playlist) 

## checking list length 

In [ ]:
first_playlist[33]['track']

In [ ]:
## checking elements as requested 

playlist["items"][33]["track"]["id"]

In [ ]:
len(playlist["items"][33]['track']['artists'][0]["name"])

In [ ]:
playlist["items"][33]['track']['artists'][0]

In [ ]:
def getSongInfo(playlist):
    all_info = []
    i=0

    for i in range(len(playlist)):
        song_dict = {}
        songtitle = []
        songuri = []
        artists = []
        audio_features = []
        songtitle.append(playlist[i]['track']["name"]) 
        songuri.append(playlist[i]['track']["id"]) 
        artists.append(playlist[i]['track']['artists'][0]["name"]) 
        audio_features.append(sp.audio_features(playlist[i]['track']["id"])) 

        song_dict['title'] = songtitle
        song_dict['artists'] = artists
        song_dict['uri'] = songuri
        song_dict["audio_features"]=audio_features
        all_info.append(song_dict)
        
    return all_info

In [ ]:
playlist1=getSongInfo(first_playlist)
## extract data from dic

In [ ]:
playlist1

In [ ]:
import pandas as pd
df = pd.DataFrame(playlist1)

## create first df

In [ ]:
df

In [ ]:
df1=pd.json_normalize(df["audio_features"], 0)

### normalized to all raws level 0 

In [ ]:
df1=df1.drop(columns=['uri', 'id','type'])
## not needed 

In [ ]:
df=df.drop(columns=['audio_features'])
## not needed 

In [ ]:
lastdf= pd.concat([df, df1], axis=1)


In [ ]:
lastdf

## last concat df with removed doubles and not relevanz columns

In [ ]:
lastdf.to_csv('songs_colldf.csv', index=False)
